In [1]:
from probttrtypes import Type, PConstraint
from utils import show, show_latex

# Judging probabilities

The witness cache in `probttr` is a pair whose first member is a list of objects and whose second member is a list of probabilities (actually probability constraints)

In [2]:
T = Type()
print(T.witness_cache)

([], [])


In [3]:
show(T.judge('a',.5))

'0.5'

In [4]:
show(T.witness_cache)

'([a], [0.5])'

In [5]:
show(T.judge('a',.6))

'0.6'

In [6]:
show(T.witness_cache)

'([a], [0.6])'

Adding an additional probability argument to `judge()` gives you a minimum and maximum probability constraint.

In [7]:
show(T.judge('a',.6,1))

'>=0.6'

In [8]:
show_latex(T.judge('a',.6,.8))

<IPython.core.display.Latex object>

In [9]:
show_latex(T.judge('a',0,.6))

<IPython.core.display.Latex object>

`judge(a,n,n)` is the same as `judge(a,n)`

In [10]:
show_latex(T.judge('a',.6,.6))

<IPython.core.display.Latex object>

Probabilities must be between 0 and 1.  For example, `show(T.judge('a',-1))` and `show(T.judge('a',.6,.1))` raise exceptions.

In contrast to non-probabilistic `pyttr` we can store a negative judgement in witness cache.

In [11]:
T.judge('a',0)
show_latex(T.witness_cache)

<IPython.core.display.Latex object>

For compatibility with non-probabilistic TTR:  `judge(a)` is the same as `judge(a,1)` (which is the same as `judge(a,1,1)`).

In [12]:
show(T.judge('a'))

'1.0'

# Querying probabilities

## Querying unconditional probabilities

If `a` is in the witness cache, `query(a)` returns the probability stored in the witness cache for `a`.

In [13]:
show(T.query('a'))

'1.0'

If `a` is not in the witness cache and we have no other way of computing a probability that `a` is a witness for `T`, then the probability range is `[0,1]`, i.e. $\leq 1$.  This corresponds to returning an answer `Don't know` or `Undecided`.

In [14]:
show(T.query('b'))

'<=1'

Witness conditions are functions which return probability constraints. Here is type `Real` for real numbers, implemented as floating point decimals where the witness condition gives a categorical judgement for any object, that is it returns probability 1 or 0.

In [15]:
def RealClassifier(n):
    if isinstance(n,float):
        return PConstraint(1)
    else:
        return PConstraint(0)
Real = Type('Real')
Real.learn_witness_condition(RealClassifier)
show(Real.query(.6))

'1.0'

In [16]:
show(Real.query('a'))

'0.0'

In [17]:
show(Real.witness_cache)

'([0.6, a], [1.0, 0.0])'

Here's an example with a couple of witness conditions which return probability constraints.  `query(a)` returns the maximum obtained by the witness conditions for `a`.  Note that this need not be identical with either of the constraints returned by the individual witness conditions since the maximum of a collection of probability constraints is defined has as minimum the maximum of all the minima and as maximum the maximum of all the maxima.  (It could, of course, be done differently...)

In [18]:
T_at = Type('T_at')
def Classifier_a(s):
    if 'a' in s:
        return PConstraint(.8,.9)
    else:
        return PConstraint(.1,.3)
def Classifier_t(s):
    if 't' in s:
        return PConstraint(.2,.95)
    else:
        return PConstraint(.15,.7)
T_at.learn_witness_condition(Classifier_a)
T_at.learn_witness_condition(Classifier_t)
show(T_at.query('a'))

'>=0.8&<=0.9'

In [19]:
show(T_at.query('t'))

'>=0.2&<=0.95'

In [20]:
show(T_at.query('at'))

'>=0.8&<=0.95'

In [21]:
show(T_at.query('b'))

'>=0.15&<=0.7'

## Querying conditional probabilities

Conditions are provided as a second argument to the `query()` method as a list each of whose members is _either_ a tuple, `(a,T)`, where `a` is an object and `T` is a type, _or_ a type.  The idea is that, for example, `T.query(a,[(b,T1),T2])` queries the probability that `a` is of type `T` given that `b` is of type `T1` and there is some witness for `T2`.  

Consider a query `T.query(a,Conds)`.  If the probability that `a` is of type `T` does not depend on any of the conditions in `Conds` then what is returned is the same as `T.query(a)`, that is the unconditional probability. The default assumption is that probabilities are independent.

In [22]:
T1 = Type()
T2 = Type()
T1.judge('a',.6)
show(T1.query('a',[('b',T2)]))

'0.6'

One kind of dependency between probabilities relates to subtyping in the type theory.  Suppose that $T_2\sqsubseteq T_1$. Then $p(a:T_1|a:T_2)=1$. A limit case of this is where we have the same type judgement in the conditions as the one we are querying: $p(a:T|a:T)$, that is, the probability that $a$ is of type $T$ given that $a$ is of type $T$ has to be $1$, no matter what the unconditional probability is that $a$ is of type $T$.

In [23]:
show(T1.query('a',[('a',T1)]))

'1.0'

Let us now create a type `T3` which is a subtype of `T1`

In [24]:
T3 = Type()
T1.learn_witness_condition(lambda x: T3.query(x))
T3.subtype_of(T1)

True

What is the probability that something is of type `T1` given that it is of type `T3`?

In [25]:
show(T1.query('b',[('b',T3)]))

'1.0'

Dependent probabilities that are not related to subtyping have to be provided by an oracle which is given as a third argument to `query()`.  An oracle is a python function which takes three arguments:  an object, a type and a list of conditions of the kind  which is used as an argument to `query()`. For any such argument it returns either a probability constraint (e.g. `PConstraint(.6)`) or `None`.  As a python function the oracle may call on resources external to `pyttr`, for example, conditional probability tables or Bayesian networks.  A call `T.query(a,c,o)` where `c` does not contain `(a,TT)` where `TT` is a subtype of `T` will return `o(a,T,c)` if this is not `None`. Otherwise `T.query(a,c,o)` will return `T.query(a)` (the unconditional probability).  If `c` _does_ contain `(a,TT)` where `TT` is a subtype of `T`, then the oracle will be ignored and `T.query(a,c,o)` will return `PConstraint(1)`.

As an example we define a trivial oracle.

In [26]:
def SillyOracle(a,T,c):
    if a is 'a'and T is T1 and ('b',T2) in c:
        return PConstraint(.7,.8)
    else:
        return 

Using the oracle.

In [27]:
show(T1.query('a',[('b',T2)],SillyOracle))

'>=0.7&<=0.8'

The oracle is not defined (returns `None`) and the result is the unconditional probability.

In [28]:
show(T1.query('a',[T2],SillyOracle))

'0.6'

The oracle is defined but is ignored because of the subtyping condition.

In [35]:
show(T1.query('a',[('b',T2),('a',T3)],SillyOracle))

'1.0'

In [30]:
import random
random.uniform(0.9999999999999999,1)

0.9999999999999999

In [31]:
len(['a'])

1

In [32]:
max([0,1])

1

In [33]:
show(PConstraint(1).max)

'1.0'